In [1]:



WITH RECURSIVE date_range AS (
  SELECT itemized_product_id, 
  DATE(start_date) AS date, 
  DATE(end_date) AS end_date
  FROM `capstone-pathlabs.1.t1` AS temp
  UNION ALL
  SELECT itemized_product_id, DATE_ADD(date, INTERVAL 1 DAY), end_date
  FROM date_range
  WHERE date < end_date
), gross_budget_agg AS (
  SELECT temp.itemized_product_id, 


------ metrics
  (AVG(p_imp)/ COUNT(DISTINCT date)) AS p_imp,
  (AVG(a_imp)/ COUNT(DISTINCT date)) AS a_imp,
  (AVG(p_clicks)/ COUNT(DISTINCT date)) AS p_clicks,
  (AVG(a_clicks)/ COUNT(DISTINCT date)) AS a_clicks,
  (AVG(p_views)/ COUNT(DISTINCT date)) AS p_views,
  (AVG(a_views)/ COUNT(DISTINCT date)) AS a_views,


------  gains and client costs


  (AVG(gross_budget)/ COUNT(DISTINCT date)) AS gross_budget,
  (AVG(net_budget)/ COUNT(DISTINCT date)) AS net_budget,
  (AVG(avg_base_cost)/ COUNT(DISTINCT date)) AS base_cost,
  (AVG(avg_spent)/ COUNT(DISTINCT date)) AS spent,
  (AVG(net_rate) / COUNT(DISTINCT date)) AS net_rate,




  ------ aggregations 
  (AVG(imp_shortfall)/ COUNT(DISTINCT date)) AS imp_shortfall,
  (AVG(clicks_shortfall)/ COUNT(DISTINCT date)) AS click_shortfall,
  (AVG(views_shortfall)/ COUNT(DISTINCT date)) AS view_shortfall,




  FROM `capstone-pathlabs.1.t1` AS temp
  JOIN date_range 
    ON date_range.itemized_product_id = temp.itemized_product_id AND date_range.date BETWEEN DATE(temp.start_date) AND DATE(temp.end_date)
  GROUP BY itemized_product_id
)
SELECT
  date_range.date,
---- identidier variables


  temp.media_plan_id,
  temp.itemized_product_id,
  temp.FlightsDimensionValueID,
  temp.LineItemsDimensionValueID,
  temp.ChannelsDimensionValueID,
  temp.PlatformsDimensionValueID,
  temp.TacticsDimensionValueID,




  ------ metrics 
  AVG(gross_budget_agg.p_imp) AS p_imp,
  AVG(gross_budget_agg.a_imp) AS a_imp,
  AVG(gross_budget_agg.p_clicks) AS p_clicks,
  AVG(gross_budget_agg.a_clicks) AS a_clicks,
  AVG(gross_budget_agg.p_views) AS p_views,
  AVG(gross_budget_agg.a_views) AS a_views ,


  ----- gains and client costs : cte - gross_budget_agg


  gross_budget_agg.gross_budget,
  gross_budget_agg.net_budget,
  AVG(gross_budget_agg.base_cost) AS base_cost,
  AVG(gross_budget_agg.spent) AS spent,


---- margins 
AVG(temp.total_margin) AS total_margin,
AVG(temp.gross_rate) AS gross_rate,
AVG(temp.net_rate) AS net_rate,
AVG(gross_budget_agg.net_rate) AS nt_rt,


temp.cost_basis AS cost_basis,


----aggragations -----
--metric price points 
AVG(p_CPM) AS p_cpm,
AVG(a_cpm) AS a_cpm,
AVG(p_cpc) AS p_cpc,
AVG(a_cpc) AS a_cpc,
AVG(p_cpv) AS p_cpv,
AVG(a_cpv) AS a_cpv,


--shortfall rates 
AVG(gross_budget_agg.imp_shortfall) AS imp_shortfall,
AVG(gross_budget_agg.click_shortfall) AS click_shortfall,
AVG(gross_budget_agg.view_shortfall) AS view_shortfall,


--over delivery rates 
AVG(imp_overdelivery_rate) AS imp_overdelivery_rate,
AVG(c_overdelivery_rate) AS c_overdelivery_rate,
AVG(v_overdelivery_rate) AS v_overdelivery_rate,


FROM date_range
JOIN `capstone-pathlabs.1.t1` AS temp 
    ON date_range.itemized_product_id = temp.itemized_product_id AND date_range.date BETWEEN DATE(temp.start_date) AND DATE(temp.end_date)
JOIN gross_budget_agg ON temp.itemized_product_id = gross_budget_agg.itemized_product_id
--WHERE CAST(temp.media_plan_id AS STRING) = '34' AND CAST(temp.FlightsDimensionValueID AS STRING) = '11133'
GROUP BY 
media_plan_id, itemized_product_id, FlightsDimensionValueID,LineItemsDimensionValueID,ChannelsDimensionValueID, PlatformsDimensionValueID, TacticsDimensionValueID,
date_range.date,gross_budget_agg.gross_budget,gross_budget_agg.net_budget,cost_basis
ORDER BY date_range.date;


ModuleNotFoundError: No module named 'pandasql'